## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Fetch id list

In [ ]:
import pandas as pd

task = 'White complex'

# IDs to remove
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]

saline_q = f"""
SELECT id, video_name, task, health, modulation, genotype, trial_length FROM dlc_table 
WHERE health = 'saline' AND modulation='NA' AND genotype='white' 
AND task IN ('ToyLight', 'FoodLight')
ORDER BY id;
"""

df = pd.read_sql_query(saline_q, conn)
saline_id = df['id'].to_list()

# Filter them out
saline_id = [i for i in saline_id if i not in bad_id]

df.to_csv("dlc_table_saline_simple.csv", index=False)

ghrelin_q = f"""
SELECT id, video_name, task, health, genotype, modulation, trial_length FROM dlc_table 
WHERE health = 'ghrelin' AND modulation='NA' AND genotype='white' 
AND task IN ('ToyLight', 'FoodLight')
ORDER BY id;
"""

df = pd.read_sql_query(ghrelin_q, conn)
ghrelin_id = df['id'].to_list()
# Filter them out
ghrelin_id = [i for i in ghrelin_id if i not in bad_id]

df.to_csv("dlc_table_ghrelin_simple.csv", index=False)

# Inh_q = f"""
# SELECT id, video_name, task, health, genotype, modulation, trial_length FROM dlc_table 
# WHERE modulation='Inhibitory'
# ORDER BY id;
# """
# df = pd.read_sql_query(Inh_q, conn)
# Inh_id = df['id'].to_list()
# Inh_id = [i for i in Inh_id if i not in bad_id]

# df.to_csv("dlc_table_inhibitory.csv", index=False)

# Exc_q = f"""
# SELECT id, video_name, task, health, genotype, modulation, trial_length FROM dlc_table 
# WHERE modulation='Excitatory'
# ORDER BY id;
# """
# df = pd.read_sql_query(Exc_q, conn)
# Exc_id = df['id'].to_list()
# Exc_id = [i for i in Exc_id if i not in bad_id]

# df.to_csv("dlc_table_excitatory.csv", index=False)

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
# print(f"Exc_id: {Exc_id}\n")
# print(f"Inh_id: {Inh_id}\n")


## Calculate time in ROI

In [ ]:
import importlib
import Python_scripts.Feature_functions.time_in_maze_regions
import Python_scripts.Data_analysis.plot_groupwise_bar
import Python_scripts.Data_analysis.compare_distributions

importlib.reload(Python_scripts.Feature_functions.time_in_maze_regions)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)
importlib.reload(Python_scripts.Data_analysis.compare_distributions)

from Python_scripts.Feature_functions.time_in_maze_regions import (
    compute_time_in_maze_regions, batch_time_in_maze_regions
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar
from Python_scripts.Data_analysis.compare_distributions import compare_distributions


## Analysis  

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

saline_time = batch_time_in_maze_regions(
    conn, saline_id, bodypart='Head', shape = 'square', 
    size = 0.5, return_fraction = True, plot_maze = False)
ghrelin_time = batch_time_in_maze_regions(
    conn, ghrelin_id, bodypart='Head', shape = 'square', 
    size = 0.5, return_fraction = True, plot_maze = False)

# Add group column to each
saline_time['group'] = 'Saline'
ghrelin_time['group'] = 'Ghrelin'
# exc_time['group'] = 'Excitatory'
# inh_time['group'] = 'Inhibitory'

# Concatenate into one DataFrame
df_time = pd.concat([saline_time, ghrelin_time], ignore_index=True)

# Reorder columns so 'group' comes first
df_time = df_time[['group', 'corner_total', 'center', 'task_specific_corner', 'trial_id']]

# Plot
fig, ax = plot_groupwise_bar(df_time, y='corner_total', ylabel='Fr. of time in corner', plot_type='bar', show_points=False)

ax.set_title(f"{task}")


In [ ]:
ax.set_ylim(0, 1)
fig.savefig(f"{task}_time_bar.pdf", dpi=300, bbox_inches='tight')


In [ ]:
from scipy.stats import ttest_ind

# Split data into groups
vals1 = df[df['group'] == 'Saline']['total_distance']
vals2 = df[df['group'] == 'Inhibitory']['total_distance']
vals3 = df[df['group'] == 'Excitatory']['total_distance']

# Welch's t-test (no assumption of equal variances)
stat, pval = ttest_ind(vals1, vals2, equal_var=False)

print(f"Welch's t-test: t = {stat:.3f}, p = {pval:.5f}")
